In [2]:
import numpy as np
import datetime
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import pandas.util.testing as tm
import os
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras import backend as K
import pickle
import cv2
import matplotlib.pyplot as plt
from keras.layers import Input
from keras.callbacks import EarlyStopping, ModelCheckpoint

C:\Users\Rajat\anaconda3\envs\tfr\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  if __name__ == '__main__':
Using TensorFlow backend.


In [ ]:
default_image_size=tuple((128, 128))
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)  
            
            return  img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
directory_root='PlantVillage'
image_list=[]
img_name=[]
list_img_filename=[]

In [ ]:
for  dirname,dis,filenames in os.walk(directory_root):
    #list_img_array.append(dirname)
    for filename in filenames:
     
        if filename.endswith(".jpg") == True or filename.endswith(".JPG") == True:
            image_list.append(convert_image_to_array(os.path.join(dirname,filename)))
            img_name.append(dirname)
            list_img_filename.append(filename)

In [ ]:
image_list[0]

In [ ]:
img_name[2]

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
label_binarizer = LabelBinarizer()
labels = label_binarizer.fit_transform(img_name)
pickle.dump(label_binarizer,open('label_trns.pkl','wb'))
n_classes1 = len(label_binarizer.classes_)

In [ ]:
n_classes1

In [ ]:
print(len(image_list))
print(len(img_name))
b=len(labels)

In [ ]:
labels[0:5]

In [ ]:
%load_ext tensorboard

In [ ]:
label_list=[]
from os import listdir
root_dir = listdir('PlantVillage')

In [ ]:
for classifi in root_dir:
    desis = os.listdir(f"{directory_root}/{classifi}")
    label_list.append(classifi)

In [ ]:
from datetime import datetime
log_dir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
X=np.array(image_list)

In [ ]:
X=X/255

In [ ]:
print("[INFO] Spliting data to train, test")
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.2,stratify=labels, random_state = 42)

In [3]:
model = tf.keras.applications.VGG16(weights="imagenet", include_top=False,input_shape=(128,128,3))

In [4]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [5]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [6]:
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

In [7]:
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x=  tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x=  tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
preds = tf.keras.layers.Dense(len(label_list), activation = 'softmax')(x)

NameError: name 'label_list' is not defined

In [ ]:
model = tf.keras.models.Model(inputs = model.input, outputs = preds)

In [8]:
print(model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [9]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [10]:
for layer in model.layers[:18]:
    layer.trainable = False

In [11]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
from datetime import datetime
log_dir="logs\\fit\\" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
hist2=model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5,callbacks=[tensorboard_callback,EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001),])

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
acc = hist2.history['accuracy']
val_acc = hist2.history['val_accuracy']
loss = hist2.history['loss']
val_loss = hist2.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()